In [17]:
from horopter.parsing.parsers import PaginatedSiteParser, parser_map
from horopter.parsing.cleaners import TextCleaner
from horopter.database import Session, Article, Task, City
from horopter.database import get_unsaved_objects
from horopter.resolver import resolve_task

import json

from importlib import reload # in order to use basicConfig with IPython
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s - %(threadName)s - %(message)s', 
                    level=logging.INFO, 
                    datefmt='%I:%M:%S')

In [19]:
#testing tasks partialy
session = Session()
parser_tasks = []
for task in session.query(Task).filter(Task.source.like('%aif.ru%')).all():
    parser_task = {
        'parser': task.parser, 
        'task': {
            'source': task.source,
            'base_url': task.base_url,
            'city_id': task.city_id,
            'parse_params': json.loads(task.parse_params),
            'fetch_params': json.loads(task.fetch_params),
            'clean_params': json.loads(task.clean_params)
        }
    }
    parser_tasks.append(parser_task)
parser_tasks[:5]

[{'parser': 'PaginatedSiteParser',
  'task': {'base_url': 'http://www.spb.aif.ru/news',
   'city_id': '7_812',
   'clean_params': {},
   'fetch_params': {'sleep': 0.25},
   'parse_params': {'text_markup': 'div.data_text a.data_anons',
    'title_markup': 'div.data_text h2.data_title a',
    'url_markup': 'div.data_text h2.data_title a'},
   'source': 'spb.aif.ru'}},
 {'parser': 'PaginatedSiteParser',
  'task': {'base_url': 'http://www.arh.aif.ru/news',
   'city_id': '7_8182',
   'clean_params': {},
   'fetch_params': {'sleep': 0.25},
   'parse_params': {'text_markup': 'div.data_text a.data_anons',
    'title_markup': 'div.data_text h2.data_title a',
    'url_markup': 'div.data_text h2.data_title a'},
   'source': 'arh.aif.ru'}},
 {'parser': 'PaginatedSiteParser',
  'task': {'base_url': 'http://www.altai.aif.ru/news',
   'city_id': '7_3852',
   'clean_params': {},
   'fetch_params': {'sleep': 0.25},
   'parse_params': {'text_markup': 'div.data_text a.data_anons',
    'title_markup': 'di

In [20]:
# custom task
parser_tasks = [
    {
        'task': {
            'base_url': 'http://www.yar.aif.ru/news',
            'city_id': '7_4852',
            'clean_params': json.loads('{}'),
            'fetch_params': json.loads('{"sleep": 0.25}'),
            'parse_params': json.loads('{"text_markup": "div.data_text a.data_anons", "title_markup": "div.data_text h2.data_title a", "url_markup": "div.data_text h2.data_title a"}'),
            'parser': 'PaginatedSiteParser',
            'source': 'yar.aif.ru'
        },
        'parser': 'PaginatedSiteParser'
    }
]

In [21]:
%%time
# run synchronously
for task in parser_tasks:
    resolve_task(task, parser_map=parser_map, verbose=False)
logging.info('Completed {} tasks'.format(len(parser_tasks)))

02:06:29 - INFO - root - MainThread - started yar.aif.ru
02:06:29 - INFO - root - MainThread - Parser for yar.aif.ru ready
02:06:30 - INFO - root - MainThread - Fetching http://www.yar.aif.ru/news
02:06:30 - INFO - root - MainThread - got 10 articles
02:06:30 - INFO - root - MainThread - 0 new articles saved in DB
02:06:30 - INFO - root - MainThread - done yar.aif.ru
02:06:30 - INFO - root - MainThread - Completed 1 tasks


Wall time: 890 ms


In [ ]:
# if something fails
session.rollback()